In [1]:
# import the essential libraries
import cv2
import numpy as np
from time import sleep
import os

In [2]:
y1 = 600
offset = 5
width = 50
height = 50
detect = []
cars = 0
capWidth = 1080
capHeight = 720
def get_center(x, y, w, h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx, cy

In [ ]:
cap = cv2.VideoCapture("Traffic - 27260.mp4")
# This class implements the Gaussian mixture model background subtraction
BGS = cv2.createBackgroundSubtractorMOG2(history=40, varThreshold=60, detectShadows=False)
forcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('cars.mp4', forcc, 30, (capWidth, capHeight))

while cap.isOpened():
    rate, frame = cap.read()
    if rate:
        frame = cv2.resize(frame, (capWidth, capHeight))
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(grayFrame, (5, 5), 5)
        img_sub = BGS.apply(blur)
        ret, thresh = cv2.threshold(img_sub, 30, 255, cv2.THRESH_BINARY)
        dilat = cv2.dilate(thresh, np.ones((8, 8)))

        contur, h = cv2.findContours(dilat, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        cv2.line(frame, (20, y1), (1050, y1), (177, 130, 40), 3)

        for i, c in enumerate(contur):
            x, y, w, h = cv2.boundingRect(c)

            if w >= width and h>= height:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                center = get_center(x, y, w, h)
                detect.append(center)
                cv2.circle(frame, center, 4, (0, 0, 255), -1)

                for x, y in detect:
                    if y < (y1+offset) and y > (y1-offset):
                        cars+=1
                        cv2.line(frame, (20, y1), (1050, y1), (0, 127, 255), 3)
                        detect.remove((x, y))
                        # print('Number of cars detected : '+str(cars))

        cv2.putText(frame, 'NUMBER OF VEHICLE : '+str(cars), (50, 70), cv2.FONT_HERSHEY_SIMPLEX,2 , (200, 50, 100), 3)
        writer.write(frame)
        # cv2.imshow('imag_sub', dilat)
        cv2.imshow('frame', frame)
        k = cv2.waitKey(40)
        if k == 27:
            break
    else:
        break
writer.release()
cap.release()
cv2.destroyAllWindows()